In [1]:
import Wrangle as w
import os
import modeling as m

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
# Machine Learning libraries
# model selection
from sklearn.model_selection import GridSearchCV
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
# helper preprocessing module

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [4]:
def get_tree(train_x, validate_x, train_y, validate_y):
    
    '''get decision tree accuracy on train and validate'''
    #create classifier then fit
    tree = DecisionTreeClassifier(max_depth=5, random_state=123)
    tree = tree.fit(train_x, train_y)

    print(f"Accuracy of Decision Tree on train data is {tree.score(train_x, train_y)}")
    print(f"Accuracy of Decision Tree on validate data is {tree.score(validate_x, validate_y)}")
    
#get_tree(train_x, validate_x, train_y, validate_y)


# ### rf

# In[24]:


def get_rf(train_x, validate_x, train_y, validate_y):

    '''get rf accuracy on train and validate'''
    #create classifier and fit
    rf = RandomForestClassifier(max_depth=5, random_state=123)
    rf = rf.fit(train_x, train_y)

    print(f"Accuracy of Random Forest on train data is {rf.score(train_x, train_y)}")
    print(f"Accuracy of Random Forest on validate data is {rf.score(validate_x, validate_y)}")


# ### regression

# In[25]:


def get_reg(train_x, validate_x, train_y, validate_y):

    '''get reg accuracy on train and validate'''
    #create classifier and fit
    logit = LogisticRegression(solver='liblinear')
    logit = logit.fit(train_x, train_y)

    print(f"Accuracy of Logistic Regression on train is {logit.score(train_x, train_y)}")
    print(f"Accuracy of Logistic Regression on validate is {logit.score(validate_x, validate_y)}")


# ### knn

# In[26]:


def get_knn(train_x, validate_x, train_y, validate_y):
    
    '''get knn accuracy on train and validate'''
    knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    knn = knn.fit(train_x, train_y)

    # print results
    print(f"Accuracy of KNN on train is {knn.score(train_x, train_y)}")
    print(f"Accuracy of KNN on validate is {knn.score(validate_x, validate_y)}")



In [5]:
df = w.get_clean_steamspy_data()

/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Publisher_{x}'] = df.publisher.str.contains(x,regex = False)
/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Publisher_{x}'] = df.publisher.str.contains(x,regex = False)
/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:62: SettingWithCopyWarning: 
A value is try

/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Genre_{x}'] = df.genre.str.contains(x)
/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Genre_{x}'] = df.genre.str.contains(x)
/Users/michaelmesa/codeup-data-science/Steam-Hours-Capstone/Michael/Wrangle.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [33]:
df.head()

,binned_hours,Publisher_Valve,Publisher_KRAFTON,Publisher__Inc.,Publisher_Rockstar_Games,Publisher_Facepunch_Studios,Publisher_Ubisoft,Publisher_Psyonix_LLC,Publisher_Bohemia_Interactive,Publisher_SEGA,...,Genre_Software Training,Genre_ Game Development,Genre_ Audio Production,Genre_ Web Publishing,Genre_Design & Illustration,Genre_Game Development,Genre_ Movie,Genre_ Education,Genre_ Software Training,Genre_Early Access
3,heavily_played,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,heavily_played,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,moderately_played,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,heavily_played,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,heavily_played,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
df.columns

Index(['appid', 'name', 'developer', 'publisher', 'positive', 'negative',
       'owners', 'average_forever', 'average_2weeks', 'median_forever',
       ...
       'Genre_Software Training', 'Genre_ Game Development',
       'Genre_ Audio Production', 'Genre_ Web Publishing',
       'Genre_Design & Illustration', 'Genre_Game Development', 'Genre_ Movie',
       'Genre_ Education', 'Genre_ Software Training', 'Genre_Early Access'],
      dtype='object', length=106)

In [7]:
train, validate, test = w.my_train_test_split(df)

In [8]:
train.binned_hours.value_counts()

moderately_played    1222
heavily_played         65
rarely_played          31
Name: binned_hours, dtype: int64

In [9]:
df, train, validate, test = m.prep_for_model(df, train, validate, test)

In [10]:
x_train, y_train, x_validate, y_validate, x_test, y_test = m.isolate_target(train,validate,test)

In [32]:
#df.columns

## Decision-Tree Modeling

In [54]:
def initialize_models():
    '''
    This function initializes a dataframe, and lists for scoring
    the models used for the project.
    '''
    depth_list = []
    sample_list = []
    c_val_list = []
    for x in range(1,21,1):
        depth_list.append(x)
    for x in range(1,21,1):
        sample_list.append(x)
    for x in range(5,100,5):
        c_val_list.append(x/100)

    
    scores = pd.DataFrame(columns=['model_name', 'train_score', 'validate_score', 'score_difference'])
    
    return scores

In [55]:
scores = initialize_models()

In [56]:
scores

,model_name,train_score,validate_score,score_difference


In [41]:
depth_list = []
sample_list = []
c_val_list = []
for x in range(1,21,1):
    depth_list.append(x)
for x in range(1,21,1):
    sample_list.append(x)
for x in range(5,100,5):
    c_val_list.append(x/100)

#%%
scores = pd.DataFrame(columns=['model_name', 'train_score', 'validate_score', 'score_difference'])

In [47]:
def get_decision_tree_results(x_train, y_train, x_validate, y_validate, scores):
    '''
    For 
    '''
    
    DTC_parameters = {'max_depth':depth_list}
    #%%
    DTC = DecisionTreeClassifier(random_state=142)
    #%%
    cv = 5
    #%%
    grid_DTC = GridSearchCV(estimator=DTC, param_grid=DTC_parameters, cv=cv, n_jobs=-1)
    #%%
    grid_DTC.fit(x_train, y_train)
    train_score = grid_DTC.best_estimator_.score(x_train, y_train)
    validate_score = grid_DTC.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'Decision Tree Parameters:  {grid_DTC.best_params_}')
    
    scores.loc[len(scores)] = ['Decision Tree', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'DTC': grid_DTC.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'DTC': grid_DTC.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.DTC, output_dict=True)))

    return scores

In [57]:
get_decision_tree_results(x_train, y_train, x_validate, y_validate, scores)

Best parameters per algorithm:
----------------------------------------------------
Decision Tree Parameters:  {'max_depth': 3}
           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.333333           0.938215            0.0  0.934091   
recall           0.058824           0.995146            0.0  0.934091   
f1-score         0.100000           0.965842            0.0  0.934091   
support         17.000000         412.000000           11.0  0.934091   

            macro avg  weighted avg  
precision    0.423849      0.891389  
recall       0.351323      0.934091  
f1-score     0.355281      0.908243  
support    440.000000    440.000000  


,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.933991,0.934091,-0.0001


In [12]:
DTC_parameters = {'max_depth':depth_list}
#%%
DTC = DecisionTreeClassifier(random_state=142)
#%%
cv = 5
#%%
grid_DTC = GridSearchCV(estimator=DTC, param_grid=DTC_parameters, cv=cv, n_jobs=-1)
#%%
grid_DTC.fit(x_train, y_train)
train_score = grid_DTC.best_estimator_.score(x_train, y_train)
validate_score = grid_DTC.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'Decision Tree Parameters:  {grid_DTC.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
Decision Tree Parameters:  {'max_depth': 3}


In [13]:
scores.loc[len(scores)] = ['Decision Tree', train_score, validate_score, train_score - validate_score]

In [14]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.933991,0.934091,-0.0001


In [15]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'DTC': grid_DTC.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'DTC': grid_DTC.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.DTC, output_dict=True)))


           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.333333           0.938215            0.0  0.934091   
recall           0.058824           0.995146            0.0  0.934091   
f1-score         0.100000           0.965842            0.0  0.934091   
support         17.000000         412.000000           11.0  0.934091   

            macro avg  weighted avg  
precision    0.423849      0.891389  
recall       0.351323      0.934091  
f1-score     0.355281      0.908243  
support    440.000000    440.000000  


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RF Modeling

In [58]:
def get_random_forest_results(x_train, y_train, x_validate, y_validate, scores):
    
    RF_parameters = {'max_depth':depth_list}
    #%%
    RF = RandomForestClassifier(random_state=142)
    #%%
    cv = 5
    #%%
    grid_RF = GridSearchCV(estimator=RF, param_grid=RF_parameters, cv=cv, n_jobs=-1)
    #%%
    grid_RF.fit(x_train, y_train)
    train_score = grid_RF.best_estimator_.score(x_train, y_train)
    validate_score = grid_RF.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'Random Forest Parameters:  {grid_RF.best_params_}')
    
    scores.loc[len(scores)] = ['Random Forest', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'RF': grid_RF.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'RF': grid_RF.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.RF, output_dict=True)))


    return scores

In [16]:
RF_parameters = {'max_depth':depth_list}
#%%
RF = RandomForestClassifier(random_state=142)
#%%
cv = 5
#%%
grid_RF = GridSearchCV(estimator=RF, param_grid=RF_parameters, cv=cv, n_jobs=-1)
#%%
grid_RF.fit(x_train, y_train)
train_score = grid_RF.best_estimator_.score(x_train, y_train)
validate_score = grid_RF.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'Random Forest Parameters:  {grid_RF.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
Random Forest Parameters:  {'max_depth': 14}


In [17]:
scores.loc[len(scores)] = ['Random Forest', train_score, validate_score, train_score - validate_score]

In [18]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.933991,0.934091,-0.000100
1,Random Forest,0.952200,0.931818,0.020382


In [19]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'RF': grid_RF.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'RF': grid_RF.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.RF, output_dict=True)))



           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.333333           0.940092            0.0  0.931818   
recall           0.117647           0.990291            0.0  0.931818   
f1-score         0.173913           0.964539            0.0  0.931818   
support         17.000000         412.000000           11.0  0.931818   

            macro avg  weighted avg  
precision    0.424475      0.893147  
recall       0.369313      0.931818  
f1-score     0.379484      0.909879  
support    440.000000    440.000000  


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KNN modeling

In [59]:
def get_knn_results(x_train, y_train, x_validate, y_validate, scores):
    
    KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
    #%%
    KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    #%%
    cv = 5
    #%%
    grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
    #%%
    grid_KNN.fit(x_train, y_train)
    train_score = grid_KNN.best_estimator_.score(x_train, y_train)
    validate_score = grid_KNN.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'KNN Parameters:  {grid_KNN.best_params_}')
    
    scores.loc[len(scores)] = ['KNN', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.KNN, output_dict=True)))


    return scores

In [20]:
KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
#%%
KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform') #KNearestNeighbors(random_state=142)
#%%
cv = 5
#%%
grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
#%%
grid_KNN.fit(x_train, y_train)
train_score = grid_KNN.best_estimator_.score(x_train, y_train)
validate_score = grid_KNN.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'KNN Parameters:  {grid_KNN.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
KNN Parameters:  {'n_neighbors': 5}


In [21]:
KNN.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [22]:
scores.loc[len(scores)] = ['KNN', train_score, validate_score, train_score - validate_score]

In [23]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.933991,0.934091,-0.000100
1,Random Forest,0.952200,0.931818,0.020382
2,KNN,0.935508,0.940909,-0.005401


In [24]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.KNN, output_dict=True)))


           heavily_played  moderately_played  rarely_played  accuracy  \
precision        1.000000           0.940639            0.0  0.940909   
recall           0.117647           1.000000            0.0  0.940909   
f1-score         0.210526           0.969412            0.0  0.940909   
support         17.000000         412.000000           11.0  0.940909   

            macro avg  weighted avg  
precision    0.646880      0.919417  
recall       0.372549      0.940909  
f1-score     0.393313      0.915856  
support    440.000000    440.000000  


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LogReg Modeling

In [60]:
def get_log_reg_results(x_train, y_train, x_validate, y_validate, scores):
    
    LR_parameters = {'C':c_val_list}
    #%%
    LR = LogisticRegression(solver='liblinear')
    #%%
    cv = 5
    #%%
    grid_LR = GridSearchCV(estimator=LR, param_grid=LR_parameters, cv=cv, n_jobs=-1)
    #%%
    grid_LR.fit(x_train, y_train)
    train_score = grid_LR.best_estimator_.score(x_train, y_train)
    validate_score = grid_LR.best_estimator_.score(x_validate, y_validate)
    #%%
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'LR Parameters:  {grid_LR.best_params_}')
    
    scores.loc[len(scores)] = ['LR', train_score, validate_score, train_score - validate_score]
    
    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'LR': grid_LR.predict(x_train)})
    y_preds_validate = pd.DataFrame({
        'y_act': y_validate,
        'baseline': 92,
        'LR': grid_LR.predict(x_validate)})
    print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.LR, output_dict=True)))



    return scores

In [25]:
LR_parameters = {'C':c_val_list}
#%%
LR = LogisticRegression(solver='liblinear') #KNearestNeighbors(random_state=142)
#%%
cv = 5
#%%
grid_LR = GridSearchCV(estimator=LR, param_grid=LR_parameters, cv=cv, n_jobs=-1)
#%%
grid_LR.fit(x_train, y_train)
train_score = grid_LR.best_estimator_.score(x_train, y_train)
validate_score = grid_LR.best_estimator_.score(x_validate, y_validate)
#%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'LR Parameters:  {grid_LR.best_params_}')

Best parameters per algorithm:
----------------------------------------------------
LR Parameters:  {'C': 0.9}


In [26]:
LR.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [27]:
scores.loc[len(scores)] = ['LR', train_score, validate_score, train_score - validate_score]

In [28]:
scores

,model_name,train_score,validate_score,score_difference
0,Decision Tree,0.933991,0.934091,-0.000100
1,Random Forest,0.952200,0.931818,0.020382
2,KNN,0.935508,0.940909,-0.005401
3,LR,0.931715,0.938636,-0.006922


In [29]:
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'LR': grid_LR.predict(x_train)})
y_preds_validate = pd.DataFrame({
    'y_act': y_validate,
    'baseline': 92,
    'LR': grid_LR.predict(x_validate)})
print(pd.DataFrame(classification_report(y_preds_validate.y_act, y_preds_validate.LR, output_dict=True)))



           heavily_played  moderately_played  rarely_played  accuracy  \
precision        1.000000           0.938497            0.0  0.938636   
recall           0.058824           1.000000            0.0  0.938636   
f1-score         0.111111           0.968273            0.0  0.938636   
support         17.000000         412.000000           11.0  0.938636   

            macro avg  weighted avg  
precision    0.646166      0.917410  
recall       0.352941      0.938636  
f1-score     0.359795      0.910948  
support    440.000000    440.000000  


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KNN TEST

In [79]:
def get_knn_test(x_train, y_train, x_test, y_test):
    
    
    scores = pd.DataFrame(columns=['model_name', 'train_score', 'test_score', 'score_difference'])
    
    KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
    
    KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    
    cv = 5
    
    grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
    
    grid_KNN.fit(x_train, y_train)
    train_score = grid_KNN.best_estimator_.score(x_train, y_train)
    test_score = grid_KNN.best_estimator_.score(x_test, y_test)
    
    print(f'Best parameters per algorithm:')
    print('----------------------------------------------------')
    print(f'KNN Parameters:  {grid_KNN.best_params_}')

    scores.loc[len(scores)] = ['KNN', train_score, test_score, train_score - test_score]

    y_preds_train = pd.DataFrame({
        'y_act': y_train,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_train)})
    y_preds_test = pd.DataFrame({
        'y_act': y_test,
        'baseline': 92,
        'KNN': grid_KNN.predict(x_test)})
    print(pd.DataFrame(classification_report(y_preds_test.y_act, y_preds_test.KNN, output_dict=True)))
    
    return scores

In [75]:
scores = pd.DataFrame(columns=['model_name', 'train_score', 'test_score', 'score_difference'])

In [76]:
scores

,model_name,train_score,test_score,score_difference


In [77]:
KNN_parameters = {'n_neighbors':[1,2,3,4,5]}
#%%
KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform')
#%%
cv = 5
#%%
grid_KNN = GridSearchCV(estimator=KNN, param_grid=KNN_parameters, cv=cv, n_jobs=-1)
#%%
grid_KNN.fit(x_train, y_train)
train_score = grid_KNN.best_estimator_.score(x_train, y_train)
test_score = grid_KNN.best_estimator_.score(x_test, y_test)
    #%%
print(f'Best parameters per algorithm:')
print('----------------------------------------------------')
print(f'KNN Parameters:  {grid_KNN.best_params_}')
    
scores.loc[len(scores)] = ['KNN', train_score, test_score, train_score - test_score]
    
y_preds_train = pd.DataFrame({
    'y_act': y_train,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_train)})
y_preds_test = pd.DataFrame({
    'y_act': y_test,
    'baseline': 92,
    'KNN': grid_KNN.predict(x_test)})
print(pd.DataFrame(classification_report(y_preds_test.y_act, y_preds_test.KNN, output_dict=True)))

Best parameters per algorithm:
----------------------------------------------------
KNN Parameters:  {'n_neighbors': 5}
           heavily_played  moderately_played  rarely_played  accuracy  \
precision        0.500000           0.924658            0.0  0.922727   
recall           0.050000           0.997537            0.0  0.922727   
f1-score         0.090909           0.959716            0.0  0.922727   
support         20.000000         406.000000           14.0  0.922727   

            macro avg  weighted avg  
precision    0.474886      0.875934  
recall       0.349179      0.922727  
f1-score     0.350208      0.889688  
support    440.000000    440.000000  


In [78]:
scores

,model_name,train_score,test_score,score_difference
0,KNN,0.935508,0.922727,0.012781
